# 挑战项目：机器人小Y
* Hi 我是Yeslab的教学机器人小Y。本项目是根据真实机器人开发过程的一种极简化模拟，虽不涉及SLAM等具体机器人算法，但以你仍然需要以逻辑形式来将我构建为一个“智能”机器人。
* 项目将分为两个阶段进行：
 - 在第一阶段，你需要按照已有代码学习并了解我的基本构造，这会包含建立地图、搭建机器人模型、完成测试等，并且在这一阶段你需要亲手设计与我的互动过程。
 - 当第一阶段顺利完成后，你就可以自己个性化的建立属于你自己的机器人，完全的个性化是第二阶段的目标，建立完成后不要忘记形成描述文档，来描述你的机器人哦。
 
* 最后，祝你玩的愉快！

![Baymax](pic/Y.png)

## 机器人建立过程
* 建立地图；
* 建立地图上的互动项目；
* 建立基本的机器人；
* 建立机器人对地图的感知程序；
* 建立路径规划程序并进行测试；
* 在地图上标记互动区域；
* 完成剧情与游戏建设；
* 编写测试。

In [1]:
import numpy as np

### 建立地图
* 地图由13x13的格子组成；
* 4个角落的桌子各占2x2的位置；
* 建立矩阵来充当地图,数字0表示空位，数字1表示被占据,在接下来的机器人模型中，数字2将用来表示机器人的位置；
* 可以在阶段二中按自己的需求来进行标记。

![robot1](pic/robot_1.png)


In [2]:
def create_map(map_lenth, table_lenth):
    # 建立地图函数
    map = np.zeros((map_lenth,map_lenth))
    map[0:table_lenth , 0:table_lenth] = 1
    map[(map_lenth-table_lenth):map_lenth , 0:table_lenth] = 1
    map[0:table_lenth ,(map_lenth-table_lenth):map_lenth] = 1
    map[(map_lenth-table_lenth):map_lenth , (map_lenth-table_lenth):map_lenth] = 1
    return map

# 测试生成地图
create_map(9,2)

array([[1., 1., 0., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 0., 1., 1.]])

### 建立互动游戏
四个角落有四个模拟工作台，当机器人靠近某一个工作台时，代表其开始完成这个工作台上的游戏，工作台周围区域应该设计为靠近区域，本实战中仅在四个工作台周围各指定一个点作为工作点。

![robot2](pic/robot_2.png)

* 左上角字符串拼接互动；
* 右上角函数绘图互动；
* 左下角矩阵乘法计算互动；
* 右下角打开文件互动；
* 为每个互动定义坐标点。

In [3]:
def service_list(game):
    # 游戏清单函数
    locA = (2,2)
    locB = (10,2)
    locC = (2,10)
    locD = (10,10)
    locE = (12,6)
    return locA,locB,locC,locD,locE

In [4]:
# Todo: 根据你自己的构思，完成每个游戏的内容

def load_gameA():
    sentence = input("\n您出上句，我对下句：\n")
    if sentence == "人生得意须尽欢":
        print("莫使金樽空对月")
        print("\ndrink!drink!drink!\n")
    else:
        print("\n大侠厉害，小的无言以对！\n")
def load_gameB():
    pass
def load_gameC():
    pass
def load_gameD():
    pass

### 建立机器人
* 建立机器人；
* 机器人需要有一个初始化点，我们设计在中心点（6,6）的位置，机器人每次完成任务后，需清理地图并回到初始点；

#### 机器人感知与路径规划
* 机器人需要有一个获取自己位置的方法；
* 机器人需能对地图上的任一点进行长度测量；
* 机器人需按照测量结果规划合适路径到达目的地。

#### 逻辑思维
* 机器人需要根据用户输入来判断是玩哪个游戏；
* 引导用户进行游戏，并不要忘记每次游戏后还原初始状态。

#### 设计一个下方中央的大门指引，方便用户开始游戏！

![robot3](pic/robot_3.png)


In [5]:
class robot():
    
    def __init__(self, name, map_lenth, table_lenth, game):
        # 存放机器人名字
        self.name = name
        # 存放游戏列表
        self.game = game
        # 存放地图
        self.robot_map = create_map(map_lenth, table_lenth)
        # 存放机器人的位置
        self.local = (int(map_lenth/2) , int(map_lenth/2))
        # 初始化机器人位置在中心
        self.robot_map[self.local] = 2
        locA,locB,locC,locD,locE = service_list(game)
        self.memory = {'A':locA, 'B':locB, 'C':locC, 'D':locD, 'E':locE}
        self.message = ""
        print(self.robot_map)
        print("\n机器人{}已就绪，输入hi进行互动".format(self.name))
        
    def localization(self):
        # 机器人定位，查看当前机器人的位置
        location = np.where(self.robot_map == 2)
        robot_x = location[0][0]
        robot_y = location[1][0]
        return (robot_x, robot_y)
    
    def sensor(self, robot_local, aim_point):
        # 机器人向目标测距
        robot_path_x = (min(robot_local[0], aim_point[0]), max(robot_local[0], aim_point[0]))
        robot_path_y = (min(robot_local[1], aim_point[1]), max(robot_local[1], aim_point[1]))
        print("\n机器人{}已获得测量结果".format(self.name))
        return (robot_path_x, robot_path_y)
    
    def move(self, robot_path, aim_point):
        # 清理地图
        self.robot_map = create_map(map_lenth, table_lenth)
        # 机器人行动逻辑
        x, y = robot_path[0], robot_path[1]
        print("\n正在前往目标点({},{})".format(aim_point[0],aim_point[1]))
        ## 当前位置写为0
        self.robot_map[self.local] = 0
        ## 沿 Y 轴纵移
        if y[0] == self.local[1]:
            self.robot_map[x[0]:x[1]+1 , y[0]] = 1
        else:
            self.robot_map[x[0]:x[1]+1 , y[1]] = 1
        ## 沿 X 轴横移
        if x[0] == self.local[1]:
            self.robot_map[x[1] , y[0]:y[1]+1] = 1
        else:
            self.robot_map[x[0] , y[0]:y[1]+1] = 1
        ## 目标位置写为2
        self.robot_map[aim_point] = 2
        print("\n位置计算完毕，机器人{}正在前进".format(self.name))
        print(self.robot_map)
        
    def go_back(self):
        self.robot_map = create_map(map_lenth, table_lenth)
        self.local = (int(map_lenth/2) , int(map_lenth/2))
        self.robot_map[self.local] = 2
        print("\n机器人{}已经回到初始位置了\n".format(self.name))
        print(self.robot_map)
    def send_service(self):
        print("A：{}\n".format(self.game[0]))
        print("B：{}\n".format(self.game[1]))
        print("C：{}\n".format(self.game[2]))
        print("D：{}\n".format(self.game[3]))
        game_choice = input("请输入字母 A、B、C、D 中的一个：")
        return game_choice
    
    def think(self, game_choice):
        robot_local = self.localization()
        robot_path = self.sensor(robot_local, self.memory[game_choice])
        self.move(robot_path, self.memory[game_choice])
        if game_choice == 'E':
            print("\nHi Boss!\n")
            print("我来服务了！请选择想玩的互动游戏，直接输入字母编号即可：\n")
        elif game_choice == 'A':
            load_gameA()
        elif game_choice == 'B':
            load_gameB()
        elif game_choice == 'C':
            load_gameC()
        elif game_choice == 'D':
            load_gameD()
        else:
            pass

### 主程序
* 设定一些参数；
* 建立main()函数来激活你的Yeslab机器人吧！
* 不要忘记对用户输入进行一些判断，拒绝不正当的指令。

In [6]:
name = '小Y'
map_lenth = 13
table_lenth = 2
game  = ["字符串拼接", "函数绘图","矩阵计算","打开文件","Hi"]

In [9]:
def main(name, map_lenth, table_lenth, game):
    my_robot = robot(name, map_lenth, table_lenth, game)
    Flag = True
    # 第一声问候
    while Flag:
        my_robot.message = input("您好，请对我说声 hi : ")
        if my_robot.message == 'hi':
            my_robot.think('E')
            game_choice = 0
            while Flag:
                game_choice = my_robot.send_service()
                if game_choice not in ['A','B','C','D']:
                    game_choice = print("您输入的指令不正确，请重新输入。\n")
                else:
                    Flag = False
        else:
            print("请输入正确的指令：hi")
       
    # 玩游戏
    my_robot.think(game_choice)
    my_robot.go_back()
    print("\nBye!\n")

In [10]:
main(name, map_lenth, table_lenth, game)

[[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]]

机器人小Y已就绪，输入hi进行互动
您好，请对我说声 hi : hi

机器人小Y已获得测量结果

正在前往目标点(12,6)

位置计算完毕，机器人小Y正在前进
[[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.

## 接下来的工作
接下来就到你了，你需要完成以下工作：
* 补完游戏内容，并进行测试；
* 备份本文档后，修改机器人模型、地图或其他你想修改的东西，按你的思维创造个性化的机器人与更多互动项目；
* 确保你的机器人游戏运行正常，制作机器人的描述文档。

### 后记
能完成此项工作，说明你很有天分而且非常努力，恭喜你。你可能会发现，机器人模型中的路径规划理解与修改会比较头疼，然而真实环境中的机器人处理路径规划也是一项不易的工作，这些挑战将包含在3D的世界、概率性的寻路以及复杂与持续变化的场景。希望你能通过本实战项目，喜欢上使用 Python 来为生活设计更多更好玩的程序。